# code for calculating researcher needs

In [1]:
import pandas
import pygsheets
import numpy

In [54]:
credentials_directory = '/Users/baird/Dropbox/_google-api/'
gc = pygsheets.authorize(service_account_env_var='GDRIVE_API_CREDENTIALS')
spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek')

#spreadsheet[1] "Gas Pipelines" tab is the second index
gas_pipes = spreadsheet.worksheet('title','Gas pipelines').get_as_df(start='A2')
oil_pipes = spreadsheet.worksheet('title', 'Oil/NGL pipelines').get_as_df(start='A2')

pipes_df_orig = gas_pipes.copy() #pandas.concat([oil_pipes, gas_pipes], ignore_index=True)

#get country ratios sheet
country_ratios_df = spreadsheet.worksheet('title', 'Country ratios by pipeline').get_as_df()

#get region defs
region_df_orig = spreadsheet.worksheet('title', 'Region dictionary').get_as_df(start='A2')

In [55]:
# replace -- entries with NaN
pipes_df_orig = pipes_df_orig.replace('--', numpy.nan)
pipes_df_orig = pipes_df_orig.loc[pipes_df_orig['PipelineName']!='']
pipes_df_orig = pipes_df_orig.loc[pipes_df_orig.Status!='N/A']
#pipes_df_orig = pipes_df_orig[pipes_df_orig['Wiki']!='']
#country_ratios_df.replace('--', numpy.nan, inplace=True)

In [56]:
status_list = ['Proposed', 'Construction', 'Shelved', 'Cancelled', 
               'Operating', 'Idle', 'Mothballed', 'Retired']
country_list = sorted(list(set(country_ratios_df['Country'])))
region_list = sorted(list(set(country_ratios_df['Region'])))

## how many oil pipelines per region?

In [57]:
pipes_df_orig.loc[pipes_df_orig.Fuel.isin(['Gas','Hydrogen'])].shape

(3212, 90)

In [58]:
projectid_count_df = pandas.DataFrame(pipes_df_orig.loc[pipes_df_orig.Fuel.isin(['Gas','Hydrogen'])].groupby(['Status','StartRegion'])['ProjectID'].count())

In [59]:
projectid_count_df.unstack().transpose().replace(numpy.nan,0)

Status                                     Cancelled  Construction  Idle  \
          StartRegion                                                      
ProjectID Australia and New Zealand              8.0           1.0   0.0   
          East Asia                             15.0          78.0   0.0   
          Eurasia                                9.0          13.0   0.0   
          Europe                                13.0          20.0   0.0   
          Latin America and the Caribbean        2.0           4.0   0.0   
          Middle East and North Africa           7.0          12.0   0.0   
          North America                         46.0          41.0   1.0   
          SE Asia                               11.0           7.0   0.0   
          South Asia                             4.0          22.0   0.0   
          Sub-Saharan Africa                     2.0           3.0   0.0   

Status                                     Mothballed  Operating  Proposed  \
          StartRegion                                                        
ProjectID Australia and New Zealand               0.0       89.0      27.0   
          East Asia                               0.0      418.0     213.0   
          Eurasia                                 7.0      171.0      80.0   
          Europe                                  4.0      636.0     160.0   
          Latin America and the Caribbean         1.0       66.0      28.0   
          Middle East and North Africa            1.0      197.0      10.0   
          North America                           0.0      275.0     102.0   
          SE Asia                                 0.0       72.0      20.0   
          South Asia                              0.0      147.0      18.0   
          Sub-Saharan Africa                      0.0       38.0      20.0   

Status                                     Retired  Shelved  
          StartRegion                                        
ProjectID Australia and New Zealand            0.0      2.0  
          East Asia                            1.0      4.0  
          Eurasia                              2.0      2.0  
          Europe                               2.0     12.0  
          Latin America and the Caribbean      0.0      8.0  
          Middle East and North Africa         1.0      4.0  
          North America                        0.0     30.0  
          SE Asia                              0.0      7.0  
          South Asia                           1.0      6.0  
          Sub-Saharan Africa                   0.0      3.0

## country-specific calculations

### for Nagwa or Arabic language expert

In [60]:
arabic_speaking_countries = ['Algeria', 
                             'Bahrain', 
                           'Chad', 
                           'Comoros', 
                           'Djibouti', 
                           'Egypt', 
                           'Iraq', 
                           'Jordan', 
                           'Kuwait', 
                           'Lebanon', 
                           'Libya', 
                           'Mauritania', 
                           'Morocco', 
                           'Oman', 
                           'Qatar', 
                           'Saudi Arabia', 
                           'Somalia', 
                           'Sudan', 
                           'Syria', 
                           'Tunisia', 
                           'United Arab Emirates', 
                           'Yemen']

In [61]:
pipes_df_orig.loc[(pipes_df_orig.StartCountry.isin(arabic_speaking_countries))].shape

(201, 90)

In [62]:
pipes_df_orig.loc[(pipes_df_orig.StartCountry.isin(arabic_speaking_countries))&
                  (pipes_df_orig.Status.isin(['Proposed','Construction','Shelved']))].shape

(16, 90)

### Russian speaking countries

In [63]:
russian_speaking_countries = ['Russia', 
                              'Belarus', 
                              'Kyrgyzstan',
                              'Kazakhstan', 
                              'Ukraine', 
                              'Azerbaijan', 
                              'Estonia', 
                              'Georgia', 
                              'Latvia', 
                              'Lithuania', 
                              'Moldova', 
                              'Tajikistan', 
                              'Turkmenistan', 
                              'Uzbekistan']

In [75]:
pipes_df_orig.loc[(pipes_df_orig.StartCountry.isin(russian_speaking_countries))].shape

(328, 90)

In [76]:
pipes_df_orig.loc[(pipes_df_orig.StartCountry.isin(russian_speaking_countries))&
                  (pipes_df_orig.Status.isin(['Proposed','Construction','Shelved']))].shape

(101, 90)

In [77]:
asia_gas_tracker_countries = [ \
                                'Bangladesh',
                                'Brunei',
                                'Cambodia',
                                #'China',
                                #'Hong Kong',
                                'India',
                                'Indonesia',
                                'Japan',
                                #'Macao',
                                'Malaysia',
                                'Myanmar',
                                'Pakistan',
                                'Philippines',
                                'Singapore',
                                'South Korea',
                                'Sri Lanka',
                                'Taiwan',
                                'Thailand',
                                'Vietnam'
                             ]

In [78]:
pipes_df_orig.loc[(pipes_df_orig.StartCountry.isin(asia_gas_tracker_countries))].shape

(428, 90)

In [79]:
pipes_df_orig.loc[(pipes_df_orig.StartCountry.isin(asia_gas_tracker_countries))&
                  (pipes_df_orig.Status.isin(['Proposed','Construction','Shelved']))].shape

(82, 90)

### China/Taiwan/Hong Kong/Macao

In [80]:
pipes_df_orig.loc[(pipes_df_orig.StartCountry.isin(['China','Taiwan','Hong Kong','Macao']))].shape

(617, 90)

In [81]:
pipes_df_orig.loc[(pipes_df_orig.StartCountry.isin(['China','Taiwan','Hong Kong','Macao']))&
                  (pipes_df_orig.Status.isin(['Proposed','Construction','Shelved']))].shape

(293, 90)

### Africa gas tracker

In [82]:
africa_gas_tracker_countries = region_df_orig.loc[region_df_orig.AfricaGasTracker=='Yes'].Country.tolist()

In [85]:
pipes_df_orig.loc[(pipes_df_orig.StartCountry.isin(africa_gas_tracker_countries))].shape

(152, 90)

In [86]:
pipes_df_orig.loc[(pipes_df_orig.StartCountry.isin(africa_gas_tracker_countries))&
                  (pipes_df_orig.Status.isin(['Proposed','Construction','Shelved']))].shape

(31, 90)